# The Data Project (BCG group)

The Data Project is structured as follows:

A. Constructing the dataset:
1. Setup
2. Importing, cleaning and merging all the needed datasets
3. Saving the created dataset (thedata)

B. Analyze and visualize the dataset:
1. Importing and presenting some summary statistics
2. Interactive plot (selection of two countries and one variable)
3. Interactive plot (selection of one country and two variables)
4. Interactive Phillips Curve (selection of which country is depicted)

# A. Constructing our dataset

# 1. Setup

At the beginning all the necessary packages are imported. 

**Note:** If you don't have datareader on your computer, you have to install it. Therefore, run at your Anaconda Prompt the command 
`pip install pandas-datareader` and `pip install wbdata` 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import wb
import ipywidgets as widgets

# 2. Importing, cleaning and merging all the needed datasets

The first dataset contains data about GDP. It is downloaded directly from the World Bank, it is cleaned and ready to be merged.

In [2]:
gdp = wb.download(indicator='NY.GDP.MKTP.KD.ZG', country=['all'], start=1997, end=2018)
gdp = gdp.rename(columns={'NY.GDP.MKTP.KD.ZG':'gdp growth'})
gdp = gdp.reset_index()
gdp['year'] = gdp['year'].astype(int) #converting column 'year' from object type to integer type
print(gdp)

                     country  year  gdp growth
0                 Arab World  2018         NaN
1                 Arab World  2017    0.976783
2                 Arab World  2016    3.242399
3                 Arab World  2015    3.342473
4                 Arab World  2014    2.452966
5                 Arab World  2013    3.135112
6                 Arab World  2012    6.695718
7                 Arab World  2011    3.608109
8                 Arab World  2010    4.749403
9                 Arab World  2009    0.465712
10                Arab World  2008    5.761859
11                Arab World  2007    4.534978
12                Arab World  2006    6.483952
13                Arab World  2005    5.720967
14                Arab World  2004    9.336038
15                Arab World  2003    5.307730
16                Arab World  2002    0.631930
17                Arab World  2001    1.623529
18                Arab World  2000    5.463191
19                Arab World  1999    1.851039
20           

**Note:** the rest of the datasets were downloaded from OECD and imported manually. 

The second dataset contains data about the Country Code [this column is used as a second parameter (the first is the column named "year") for the final merg].

In [3]:
country_codes = "./dataproject/data/ccode.xls"
ccode = pd.read_excel(country_codes)
ccode.rename(columns = {'Country':'country', 'CODE':'Country Code'}, inplace = True)
ccode.head()

,Country Code,country
0,ABW,Aruba
1,AFG,Afghanistan
2,AFRI,Africa
3,AGO,Angola
4,AIA,Anguilla


The first two datasets are merged. 

In [4]:
# Merging the two imported DataFrames (ccode and gdp)
GDP_ccode = pd.merge(gdp, ccode, how = 'outer', on = ['country'])
GDP_ccode.rename(columns = {'year':'Year'}, inplace = True)
GDP_ccode['Country Code'].unique()
GDP_ccode.head()

,country,Year,gdp growth,Country Code
0,Arab World,2018.0,NaN,NaN
1,Arab World,2017.0,0.976783,NaN
2,Arab World,2016.0,3.242399,NaN
3,Arab World,2015.0,3.342473,NaN
4,Arab World,2014.0,2.452966,NaN


The third dataset is imported, which contains data about the Average Wage. The unnecessary columns are dropped, while the remaining are renamed and the Averafe Wage Growth (%) is calculated and 

In [5]:
avg_wage = "./dataproject/data/OECDwage.csv"
wages = pd.read_csv(avg_wage)
wages.head()

# Cleaning the dataset, calculate the Average Wage Growth (%) and store it in a new column
drop_these = ['INDICATOR', 'SUBJECT' ,'MEASURE', 'FREQUENCY', 'Flag Codes']
wages.drop(drop_these, axis = 1, inplace = True)

wages.rename(columns = {'LOCATION':'Country Code', 'TIME':'Year', 'Value':'Average Wage'}, inplace = True)
wages["Average Wage Growth (%)"] = wages['Average Wage'].pct_change()*100
wages.head()

,Country Code,Year,Average Wage,Average Wage Growth (%)
0,AUS,1997,40902.825366,NaN
1,AUS,1998,41503.236390,1.467896
2,AUS,1999,42556.169985,2.536992
3,AUS,2000,42702.425882,0.343677
4,AUS,2001,42311.078226,-0.916453


Another dataset is imported with data about Total Unemployment (%). The same procedure is followed, so as to keep only the desired columns.

In [6]:
total_unemployment = "./dataproject/data/unempl.csv"
tot_unempl = pd.read_csv(total_unemployment)

# Cleaning and manipulating the data
drop_these = ['INDICATOR', 'SUBJECT' , 'MEASURE', 'FREQUENCY', 'Flag Codes']
tot_unempl.drop(drop_these, axis = 1, inplace = True)
tot_unempl.rename(columns = {'LOCATION':'Country Code', 'TIME':'Year', 'Value':'Total Unemployment (%)'}, inplace = True)
tot_unempl.head()

,Country Code,Year,Total Unemployment (%)
0,AUS,1997,8.362488
1,AUS,1998,7.677429
2,AUS,1999,6.873791
3,AUS,2000,6.285546
4,AUS,2001,6.742173


Our final dataset is imported with data about Inflation Rate (%); the same procedure is followed to keep only the needed columns

In [7]:
inflation_rate = "./dataproject/data/inflation.csv"
inflation = pd.read_csv(inflation_rate)

# Cleaning and manipulating the data
drop_columnsi = ['INDICATOR', 'SUBJECT' , 'MEASURE', 'FREQUENCY', 'Flag Codes']
inflation.drop(drop_columnsi, axis = 1, inplace = True)
inflation.rename(columns = {'LOCATION':'Country Code', 'TIME':'Year', 'Value':'Inflation Rate (%)'}, inplace = True)
inflation.head()

,Country Code,Year,Inflation Rate (%)
0,AUS,1997,0.224887
1,AUS,1998,0.860135
2,AUS,1999,1.483129
3,AUS,2000,4.457435
4,AUS,2001,4.407135


The last three datasets are merged (at the beginning the first two are merged and then the one that was just created with the third dataset).

In [8]:
wage_unempl = pd.merge(wages, tot_unempl, how = 'outer', on = ['Country Code', 'Year'])
wage_unempl.head(10)

wage_unempl_infl = pd.merge(wage_unempl, inflation, how = 'outer', on = ['Country Code','Year'])
wage_unempl_infl.head()

,Country Code,Year,Average Wage,Average Wage Growth (%),Total Unemployment (%),Inflation Rate (%)
0,AUS,1997,40902.825366,NaN,8.362488,0.224887
1,AUS,1998,41503.236390,1.467896,7.677429,0.860135
2,AUS,1999,42556.169985,2.536992,6.873791,1.483129
3,AUS,2000,42702.425882,0.343677,6.285546,4.457435
4,AUS,2001,42311.078226,-0.916453,6.742173,4.407135


Merging the final dataset (the one that was just merged and the one that was merged at the beginning of the assignment).

In [9]:
final = pd.merge(wage_unempl_infl, GDP_ccode, on = ['Country Code', 'Year'], how = 'left')
final.head()

# Changing the order of columns, so as to have the wanted format 
final = final.reindex(columns = ['Year','country','Country Code','gdp growth', 'Inflation Rate (%)','Total Unemployment (%)', 'Average Wage', 'Average Wage Growth (%)'])
final.head(20)

,Year,country,Country Code,gdp growth,Inflation Rate (%),Total Unemployment (%),Average Wage,Average Wage Growth (%)
0,1997,Australia,AUS,3.966420,0.224887,8.362488,40902.825366,NaN
1,1998,Australia,AUS,4.576739,0.860135,7.677429,41503.236390,1.467896
2,1999,Australia,AUS,5.018064,1.483129,6.873791,42556.169985,2.536992
3,2000,Australia,AUS,3.943189,4.457435,6.285546,42702.425882,0.343677
4,2001,Australia,AUS,1.933106,4.407135,6.742173,42311.078226,-0.916453
5,2002,Australia,AUS,4.003022,2.981575,6.368911,42602.277692,0.688235
6,2003,Australia,AUS,2.985745,2.732596,5.928420,43100.656217,1.169840
7,2004,Australia,AUS,4.001106,2.343255,5.396734,44441.813542,3.111687
8,2005,Australia,AUS,3.190316,2.691832,5.033881,45016.207678,1.292463
9,2006,Australia,AUS,2.834233,3.555288,4.785240,45307.351517,0.646753


We are only interested in OECD countries from 2003 to 2017. So, all the unnecessary observation are dropped.

In [10]:
# Dropping G20, EU28, EA19, non-OECD countries and years that are not in the range (2003, 2017)
for val in ['20','19','28']:
    I = final['Country Code'].str.contains(val)
    final = final.loc[I == False]

A = ['Brazil', 'Indonesia', 'South Africa', 'Colombia', 'China', 'India', 'Saudi Arabia', 'Argentina', 'Costa Rica']
for val in A: 
    I = final['country'].str.contains(val)
    final = final.loc[I == False]   
final['country'].unique() 

final = final[final.Year > 2002]
final = final[final.Year < 2018]

We checked for missing values in our dataset. Data at the Total Unemployment (%) column  are missing, for Switzerland (from 2003 to 2009), for Lithuania (from 2003 to 2004) and for Turkey (from 2003 to 2005). These data were retrieved from the corresponding datasets from the World Bank and imported manually to our dataset. Moreover, at the columns Average Wage and Average Wage Growth (%) data for Turkey are missing; we were unable to find these data, so we decided to leave them as NaN. 

In [11]:
final[final.isnull().any(axis=1)] # Checking for missing values in our dataset 

,Year,country,Country Code,gdp growth,Inflation Rate (%),Total Unemployment (%),Average Wage,Average Wage Growth (%)
510,2003,Switzerland,CHE,0.039577,0.638299,NaN,57167.661706,-0.256389
511,2004,Switzerland,CHE,2.776224,0.802883,NaN,56674.850849,-0.862045
512,2005,Switzerland,CHE,3.115246,1.171951,NaN,57534.599869,1.516985
513,2006,Switzerland,CHE,3.988634,1.059525,NaN,57889.093293,0.616140
514,2007,Switzerland,CHE,4.111878,0.732337,NaN,58745.681928,1.479706
515,2008,Switzerland,CHE,2.154702,2.426048,NaN,59056.552357,0.529180
516,2009,Switzerland,CHE,-2.222107,-0.480468,NaN,60255.103116,2.029497
720,2003,Lithuania,LTU,10.538565,-1.134309,NaN,11232.255947,6.644158
721,2004,Lithuania,LTU,6.550083,1.164103,NaN,12157.152833,8.234293
748,2006,Turkey,TUR,7.109756,9.597242,8.750455,NaN,NaN


In [12]:
# Adding missing values for unemployment (for Switzerland, Lithuania and Turkey), data retrieved from the World Bank.
add = {'2003':'4.119999886','2004':'4.320000172', '2005':'4.440000057', '2006':'4','2007':'3.65000009', '2008':'3.349999905','2009':'4.119999886' }
for key, value in add.items():
   I = (final['country'] == 'Switzerland') & (final['Year'] == int(key))
   final.loc[I, ['Total Unemployment (%)']] = value
final[final['country'] == 'Switzerland'].head(10)

addL = {'2003':'12.86999989','2004':'10.68000031'}
for key, value in addL.items():
   I = (final['country'] == 'Lithuania') & (final['Year'] == int(key))
   final.loc[I, ['Total Unemployment (%)']] = value

addT =  {'2003':'10.53999996','2004':'10.84000015', '2005':'10.64000034'}
for key, value in addT.items():
   I = (final['country'] == 'Turkey') & (final['Year'] == int(key))
   final.loc[I, ['Total Unemployment (%)']] = value

final = final.sort_values(['country','Year'])
final[final.isnull().any(axis=1)] #Checking again if there are any NaN values left in our dataset
                                  #(NaN for Turkey - Average Wage and Average Wage Growth (%))

,Year,country,Country Code,gdp growth,Inflation Rate (%),Total Unemployment (%),Average Wage,Average Wage Growth (%)
915,2003,Turkey,TUR,5.608342,21.602440,10.53999996,NaN,NaN
916,2004,Turkey,TUR,9.644246,8.598262,10.84000015,NaN,NaN
917,2005,Turkey,TUR,9.009901,8.179160,10.64000034,NaN,NaN
748,2006,Turkey,TUR,7.109756,9.597242,8.75046,NaN,NaN
749,2007,Turkey,TUR,5.030441,8.756181,8.88715,NaN,NaN
750,2008,Turkey,TUR,0.845219,10.444130,9.72854,NaN,NaN
751,2009,Turkey,TUR,-4.704445,6.250977,12.5806,NaN,NaN
752,2010,Turkey,TUR,8.487358,8.566444,10.6845,NaN,NaN
753,2011,Turkey,TUR,11.113499,6.471879,8.81476,NaN,NaN
754,2012,Turkey,TUR,4.789920,8.891570,8.16661,NaN,NaN


Doing some last manipulations to our dataset, so as the columns' names and the observations (rounded to two decimals) have the wanted format.

In [13]:
# Converting "Unemployment Rate (%)" column to float type
final['Total Unemployment (%)'] = final['Total Unemployment (%)'].astype(float)
final.info()

final.rename(columns = {'country':'Country', 'gdp growth':'GDP Growth (%)'}, inplace = True) # Renaming "country" and "gdp growth" for consistency

final['GDP Growth (%)'] = final['GDP Growth (%)'].round(2)
final['Inflation Rate (%)'] = final['Inflation Rate (%)'].round(2)
final['Total Unemployment (%)'] = final['Total Unemployment (%)'].round(2)
final['Average Wage'] = final['Average Wage'].round(2)
final['Average Wage Growth (%)'] = final['Average Wage Growth (%)'].round(2)
final.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540 entries, 6 to 566
Data columns (total 8 columns):
Year                       540 non-null int64
country                    540 non-null object
Country Code               540 non-null object
gdp growth                 540 non-null float64
Inflation Rate (%)         540 non-null float64
Total Unemployment (%)     540 non-null float64
Average Wage               525 non-null float64
Average Wage Growth (%)    525 non-null float64
dtypes: float64(5), int64(1), object(2)
memory usage: 38.0+ KB


# 3. Saving the created dataset (thedata) 

The goal is to to work with our dataset. This way we can import directly and start working with it, which saves time. 

In [14]:
final.to_csv("./dataproject/data/thedata.csv", index = False)

# B. Analyze and visualize the dataset

# 1(i). Importing the dataset

In [3]:
# Importing the dataset and renaming the columns
thedata = pd.read_csv("./dataproject/data/thedata.csv")
thedata.rename(columns = {'country':'Country', 'gdp growth':'GDP Growth (%)', '(%) AVG Wage':'AVG Wage Growth (%)'}, inplace = True)
thedata.head()

,Year,Country,Country Code,GDP Growth (%),Inflation Rate (%),Total Unemployment (%),Average Wage,Average Wage Growth (%)
0,2003,Australia,AUS,2.99,2.73,5.93,43100.66,1.17
1,2004,Australia,AUS,4.00,2.34,5.40,44441.81,3.11
2,2005,Australia,AUS,3.19,2.69,5.03,45016.21,1.29
3,2006,Australia,AUS,2.83,3.56,4.79,45307.35,0.65
4,2007,Australia,AUS,3.78,2.33,4.38,46557.62,2.76


# 1(ii). Presenting some summary statistics of our dataset

In [9]:
thedata.groupby(('Country')).mean()

,Year,GDP Growth (%),Inflation Rate (%),Total Unemployment (%),Average Wage,Average Wage Growth (%)
Country,,,,,,
Australia,2010,2.873333,2.495333,5.328667,47561.849333,0.965333
Austria,2010,1.512000,1.875333,5.159333,48621.005333,0.646000
Belgium,2010,1.454000,1.946000,7.927333,49430.102000,0.088667
Canada,2010,1.972667,1.790667,7.033333,44165.309333,1.542000
Chile,2010,3.893333,3.224667,7.638667,23619.280000,1.602667
Czech Republic,2010,2.870000,1.960000,6.233333,22101.700667,2.506000
Denmark,2010,1.106667,1.603333,5.808000,48460.684000,1.216000
Estonia,2010,3.236000,3.160667,8.782667,20227.312667,4.125333
Finland,2010,1.202667,1.378000,8.192000,41631.775333,1.022000


**Conclusions:** After calculating the mean value for each country for these fifteen years period, we can conclude that:
1. **Turkey, Ireland and Slovak Rebublic** have the biggest average GDP Growth (%) (5.820667 , 4.750667 and 4.081333 respectively)
2. **Greece** is the only country with a negative average GDP Growth (%) (-0.524000), while **Italy and Portugal** follows (with 0.051333 and 0.322000 respectively)
3. **Turkey** faced a significant increase in the average Inflation Rate (%) (9.352667), while at the second spot is **Iceland** with a smaller increase (4.748667). 
4. **Japan and Switzerland** faced the smallest change at their average Inflation Rate (%) (0.200000 and 0.354000 respectively)
5. **Spain, Greece and Slovak Republic** faced the biggest change in terms of Total Unemployment Rate (%) (16.858000, 16.272000 and 13.114667 respectively)
6. **Korea Republic (or South Korea), Norway and Japan** faced the lowest changes in their average Total Unemployment Rate (%) (3.492667, 3.602667 and 4.156667 respectively)
7. **Lithuania, Latvia and Estonia** faced the biggest changes in their Average Wage Growth (%) (5.870667, 5.753333 and 4.125333 respectively)
8. Three countries faced negative changes in their Average Wage Growth (%), **Greece, Portugal and Mexico** (-0.923333, -0.268000 and -0.212667 respectively) 

# 2. Interactive plot (selection of two countries and one variable)

An interactive plot where, using dropdown lists, the user can choose any of the two countries from our dataset and one of the economic indicators. In this way, the user will be able to compare two countries of his/her own choice, during a 15 years period, in terms of GDP Growth (%), Inflation Rate (%), Unemployment Rate (%) or Average Wage Growth (%). 

**Note:** data for the Average Wage Growth (%) for Turkey are missing, so if 'Turkey' is chosen as country and 'Average Wage Growth (%)' as variable 1, nothing will be depicted for that country.

In [4]:
def _plot_1(thedata, Country1, Country2, Variable1):

    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    thedata.loc[:,['Year']] = pd.to_numeric(thedata['Year'])
                                                                
    I = (thedata['Country'] == Country1)
    i = (thedata['Country'] == Country2)
    
    x = thedata.loc[I,'Year']
    y = thedata.loc[I,Variable1]
    z = thedata.loc[i,Variable1]

    label1 = Country1
    label2 = Country2
    ax.plot(x, y, label = label1)
    ax.plot(x, z, label = label2)
    
    ax.set_xticks(list(range(2003, 2017 + 1, 2)))
    ax.set_xlabel('Year')
    ax.legend(loc = 'upper right')
    


def plot_1(thedata):
 
    widgets.interact(_plot_1,  
    thedata = widgets.fixed(thedata),
        Country1 = widgets.Dropdown(
        description = 'OECD Country (No data for % AVG Wage for Turkey)', 
        options = thedata['Country'].unique().tolist(),
        value = 'Australia',
        disabled = False),

        Country2 = widgets.Dropdown(
        description = 'OECD Country (No data for % AVG Wage for Turkey)', 
        options = thedata['Country'].unique().tolist(),
        value = 'Australia',
        disabled = False),

        Variable1 = widgets.Dropdown(
        description = 'Variable1', 
        options = ['Total Unemployment (%)','Inflation Rate (%)','Average Wage Growth (%)','GDP Growth (%)'], 
        value = 'Total Unemployment (%)'),

    )    
plot_1(thedata)

interactive(children=(Dropdown(description='OECD Country (No data for % AVG Wage for Turkey)', options=('Austr…

**Conclusion:** All 

# 3. Interactive plot (selection of one country and two variables)

The second plot uses dropdown lists, with which you can select two indicators for the same country. This provides to the user very nice insights about the evolution and the relationship of the two selected indicators within a country and let him/her answer questions like: "What grew faster during the last 15 years, the Average Wage or the Inflation?" or "Does Unemployment Rate depend on GDP Growth?" etc.

**Note:** data for the Average Wage Growth (%) for Turkey are missing, so if 'Turkey' is chosen as country and 'Average Wage Growth (%)' as variable 1 or variable 2, nothing will be depicted for that country.

In [5]:
def _plot_2(thedata, Country, variable1, variable2):

    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    thedata.loc[:,['Year']] = pd.to_numeric(thedata['Year'])
                                                                
    I = (thedata['Country'] == Country)
    
    x = thedata.loc[I,'Year']
    y = thedata.loc[I,variable1]
    z = thedata.loc[I,variable2]
    ax.plot(x, y, 'g')
    ax.plot(x, z, 'y')
    
    ax.set_xticks(list(range(2003, 2017 + 1, 2)))
    ax.set_xlabel('Year')
    ax.legend(loc = 'upper right')

def plot_2(thedata):
    
    widgets.interact(_plot_2,  
    thedata = widgets.fixed(thedata),
        Country = widgets.Dropdown(
        description = 'OECD Country (No data for % AVG Wage for Turkey)', 
        options = thedata['Country'].unique().tolist(),
        value = 'Australia',
        disabled = False),
                     
        variable1 = widgets.Dropdown(
        description = 'Variable1', 
        options = ['Total Unemployment (%)','Inflation Rate (%)','Average Wage Growth (%)','GDP Growth (%)'], 
        value = 'GDP Growth (%)'),
                     
        variable2 = widgets.Dropdown(
        description = 'Variable2', 
        options = ['Total Unemployment (%)','Inflation Rate (%)','Average Wage Growth (%)','GDP Growth (%)'], 
        value = 'Inflation Rate (%)')
        
    )                 

plot_2(thedata)

interactive(children=(Dropdown(description='OECD Country (No data for % AVG Wage for Turkey)', options=('Austr…

# 4. Interactive Phillips Curve (selection of which country is depicted)

The third plot represents the famous Phillips curve and due to the plotting of the best fitted line, we can observe if there is any realtionship between Unemployment Rate (%) and Inflation Rate (%).

In [6]:
def _phillips_curve(thedata, Country):
 
    thedata.loc[:,['Year']] = pd.to_numeric(thedata['Year'])
    
    I = (thedata['Country'] == Country)
    
    a = thedata.loc[I,'Total Unemployment (%)']
    b = thedata.loc[I,'Inflation Rate (%)']
    
    #Reseting the index so as the data of the chosen country are at the top of the dataset
    #Otherwise the annotation fuction doesn't work apart from the default value ('Australia')
    a = a.reset_index()
    b = b.reset_index()
    a = a['Total Unemployment (%)']
    b = b['Inflation Rate (%)']
    
    plt.scatter(a,b)
    plt.xlabel('Total Unemployment (%)')
    plt.ylabel('Inflation Rate (%)')
    plt.title('Philips Curve')
    
    plt.plot(a, b, '--')

    YEAR = thedata.loc[I,'Year']
    
    plt.plot(np.unique(a), np.poly1d(np.polyfit(a, b, 1))(np.unique(a)))
    
    for i, txt in enumerate(YEAR):
        plt.annotate(txt, (a[i],b[i]))
    
def phillips_curve(thedata):
    
    widgets.interact(_phillips_curve,  
    thedata = widgets.fixed(thedata),
        Country = widgets.Dropdown(
        description = 'OECD Country', 
        options = thedata['Country'].unique().tolist(),
        value = 'Australia',
        disabled = False)
                    )
                    
phillips_curve(thedata)

interactive(children=(Dropdown(description='OECD Country', options=('Australia', 'Austria', 'Belgium', 'Canada…